# Importacoes

In [ ]:
import tensorflow as tf


In [ ]:
print(tf.__version__)


In [ ]:
import requests
import pandas as pd
import numpy as np


# Conectando com servidor

In [ ]:
diaInicio = '20210801'
diaFinal = '20220331'
idGrupo = '12'
idRegra = '38'
idTipo = 'null'
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(url)


In [ ]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)


In [ ]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')


In [ ]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape


In [ ]:
Y = np.stack(df['campoY'].values)
Y = Y.reshape(-1,1)
Y.shape


# Dados Validacao

In [ ]:
diaInicio = '20220401'
diaFinal = '20220531'
idGrupo = '12'
idRegra = '38'
idTipo = 'null'
urlValida = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(urlValida)


In [ ]:
r = requests.get(urlValida)
j = r.json()
dfValida = pd.DataFrame.from_dict(j)


In [ ]:
dfValida.head()


In [ ]:
dfTratadoValidaX = dfValida.apply(trataLinhaX, axis='columns')
XValida = np.stack(dfTratadoValidaX.values)
XValida.shape


In [ ]:
YValida = np.stack(dfValida['campoY'].values)
YValida = YValida.reshape(-1,1)
YValida.shape


# Minha Rede

## Estrutra

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1800,)),
    tf.keras.layers.Dense(1000, activation=tf.nn.relu),
    tf.keras.layers.Dense(500, activation=tf.nn.relu), 
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
model.summary()


## Compilacao

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=1e-4),
                metrics=['accuracy'])


## Fit - Treino

In [ ]:
history = model.fit(x=X,y=Y, validation_data=(XValida, YValida) ,
                    steps_per_epoch=100,
                    epochs=15,
                    validation_steps=50,
                    verbose=2)


## Salva Rede

In [ ]:
path='pesos/treino_rede49.h5'
model.save(path)
